Due to very low volume of data witch claim frequency above 0, I will use two different aproaches to modeling this subject:
1. GLM, zero-inflated model - the key point in this aproach is to choose proper distribution function for our variable. Due to high skewness and variance aproximatelly 2 times higher than mean I will compere Poisson and Negative Binomial distribution and choose which will better suit the data.
2. two level model - classification + regression

In [40]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

### Data transformation

In [ ]:
df = pd.read_csv("feature_selected_data.csv")
df

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,Area,VehBrand,VehGas,Region,ClaimFrequency,VehPowerDriverAge,TransformBonusMalus,TransformDensity
0,1.0,0.10000,5.0,0.0,55.0,D,B12,Regular,R82,1.000000,0.090909,3.912023,7.104144
1,1.0,0.77000,5.0,0.0,55.0,D,B12,Regular,R82,1.298701,0.090909,3.912023,7.104144
2,1.0,0.75000,6.0,2.0,52.0,B,B12,Diesel,R22,1.333333,0.115385,3.912023,3.988984
3,1.0,0.84000,7.0,0.0,46.0,B,B12,Diesel,R72,1.190476,0.152174,3.912023,4.330733
4,1.0,0.52000,6.0,2.0,38.0,E,B12,Regular,R31,1.923077,0.157895,3.912023,8.007367
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677997,0.0,0.00274,4.0,0.0,54.0,E,B12,Regular,R93,0.000000,0.074074,3.912023,8.106816
677998,0.0,0.00274,4.0,0.0,41.0,E,B12,Regular,R11,0.000000,0.097561,4.553877,9.195227
677999,0.0,0.00274,6.0,2.0,45.0,D,B12,Diesel,R82,0.000000,0.133333,3.912023,7.187657
678000,0.0,0.00274,4.0,0.0,60.0,B,B12,Regular,R26,0.000000,0.066667,3.912023,4.553877


In [ ]:
to_encode = df.select_dtypes(include=object)

ohe = OneHotEncoder(sparse_output=False, drop="first")

encoded_array = ohe.fit_transform(to_encode)
encoded_df = pd.DataFrame(encoded_array, columns=ohe.get_feature_names_out(to_encode.columns))
df_final = pd.concat([df.select_dtypes(include=np.number), encoded_df], axis=1)

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,ClaimFrequency,VehPowerDriverAge,TransformBonusMalus,TransformDensity,Area_B,...,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94,isClaim
0,1.0,0.10000,5.0,0.0,55.0,1.000000,0.090909,3.912023,7.104144,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
1,1.0,0.77000,5.0,0.0,55.0,1.298701,0.090909,3.912023,7.104144,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,1.0,0.75000,6.0,2.0,52.0,1.333333,0.115385,3.912023,3.988984,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.84000,7.0,0.0,46.0,1.190476,0.152174,3.912023,4.330733,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.52000,6.0,2.0,38.0,1.923077,0.157895,3.912023,8.007367,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677997,0.0,0.00274,4.0,0.0,54.0,0.000000,0.074074,3.912023,8.106816,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
677998,0.0,0.00274,4.0,0.0,41.0,0.000000,0.097561,4.553877,9.195227,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
677999,0.0,0.00274,6.0,2.0,45.0,0.000000,0.133333,3.912023,7.187657,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
678000,0.0,0.00274,4.0,0.0,60.0,0.000000,0.066667,3.912023,4.553877,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [29]:
df_final["isClaim"] = np.where(df_final["ClaimNb"] > 0, 1, 0)
df_final.drop(["ClaimNb", "Exposure"], axis=1, inplace=True)
df_final

,VehPower,VehAge,DrivAge,ClaimFrequency,VehPowerDriverAge,TransformBonusMalus,TransformDensity,Area_B,Area_C,Area_D,...,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94,isClaim
0,5.0,0.0,55.0,1.000000,0.090909,3.912023,7.104144,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
1,5.0,0.0,55.0,1.298701,0.090909,3.912023,7.104144,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,6.0,2.0,52.0,1.333333,0.115385,3.912023,3.988984,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,7.0,0.0,46.0,1.190476,0.152174,3.912023,4.330733,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,6.0,2.0,38.0,1.923077,0.157895,3.912023,8.007367,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677997,4.0,0.0,54.0,0.000000,0.074074,3.912023,8.106816,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
677998,4.0,0.0,41.0,0.000000,0.097561,4.553877,9.195227,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
677999,6.0,2.0,45.0,0.000000,0.133333,3.912023,7.187657,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
678000,4.0,0.0,60.0,0.000000,0.066667,3.912023,4.553877,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df_final.drop("ClaimFrequency", axis=1), df_final["ClaimFrequency"], test_size=0.2, stratify=df_final["isClaim"], random_state=42)

In [ ]:
# scaler = StandardScaler()
# scale = scaler.fit_transform(X_train)

In [41]:
# Define formula
df_GLM = df_final.drop(["isClaim"], axis=1)
formula = "ClaimFrequency ~ " + " + ".join(df_GLM.columns.difference(["ClaimFrequency"]))

# Start MLflow experiment
mlflow.set_experiment("Zero-Inflated GLM Experiment")

with mlflow.start_run():
    # Log model type
    mlflow.log_param("model_type", "GLM")
    
    # Train Poisson model
    model_pois = smf.glm(formula, data=df_GLM, family=sm.families.Poisson()).fit()
    mlflow.log_metric("AIC_Poisson", model_pois.aic)
    mlflow.log_metric("LogLik_Poisson", model_pois.llf)
    
    # Train Negative Binomial model
    model_nb = smf.glm(formula, data=df_GLM, family=sm.families.NegativeBinomial()).fit()
    mlflow.log_metric("AIC_Negative_Binomial", model_nb.aic)
    mlflow.log_metric("LogLik_Negative_Binomial", model_nb.llf)
    
    # Likelihood Ratio Test (LRT) between Poisson and NB
    lr_stat = 2 * (model_nb.llf - model_pois.llf)
    p_value = stats.chi2.sf(lr_stat, df=1)
    mlflow.log_metric("LR_Test_Stat", lr_stat)
    mlflow.log_metric("LR_p_value", p_value)
    
    # Log final model (saving the best based on AIC)
    best_model = "Negative_Binomial" if model_nb.aic < model_pois.aic else "Poisson"
    mlflow.log_param("Best_Model", best_model)

    # Save the models
    model_nb.save("nb_model.pkl")
    model_pois.save("poisson_model.pkl")
    mlflow.log_artifact("nb_model.pkl")
    mlflow.log_artifact("poisson_model.pkl")

    print(f"Poisson AIC: {model_pois.aic}, NB AIC: {model_nb.aic}")
    print(f"LR Test p-value: {p_value}")

print("MLflow experiment completed.")


d:\Projects\PricingRecrutationProject\FrequencyOfClaimsPrediction\.venv\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


Poisson AIC: 393745.3111630523, NB AIC: 368039.6831925374
LR Test p-value: 0.0
MLflow experiment completed.
